In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57e04000 @  0x7fb3ffa662a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import numpy as np
import pandas as pd

from collections import Counter

import torch
import torch.nn as nn

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [0]:
path='gdrive/My Drive/netflix_colab/data/'

# CREATING PYTORCH DATA CLASS:

In [0]:
class Recommender_Data_Classifn(Dataset):

    def __init__(self, path, csv_file_name):
        
        df = pd.read_csv(path + csv_file_name, index_col = 0)
        x  = df.iloc[:, :-1].values
        y  = df.iloc[:, -1].values - 1   #-1 becoz C.E.LOSS infers n_classes from dimensionality of y_pred as range(y_pred.shape[0])
                                         # ie: If 5 classes present then labels should belong to interval (0, 4)
        self.len = df.shape[0]
        
        self.x_data = torch.from_numpy(x).type(torch.LongTensor)
        self.y_data = torch.from_numpy(y).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
train_dataset = Recommender_Data_Classifn(path, 'df_final_trainset.csv')
test_dataset  = Recommender_Data_Classifn(path, 'df_final_testset.csv')

In [0]:
train_dataset.x_data

tensor([[    0,     0],
        [    1,     1],
        [    2,     2],
        ...,
        [ 3652,  8903],
        [   56, 10623],
        [  263, 39396]])

In [0]:
train_dataset.y_data

tensor([2, 4, 2,  ..., 0, 3, 4])

# TRAIN-EVALUATION FUNCTION:

In [0]:
def fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs):
    
    # INNER FNS---------------------------------------------
    def retrn_scalar_y_pred(y_pred):
        _, y_pred_scalar = torch.max(y_pred, dim = 1) # idxs of max vals of each y_pred
        return y_pred_scalar

    def retrn_n_matches(y_pred_scalar, target):
        n_matches = (y_pred_scalar == target).sum().item()
        return n_matches
    
    # TRAIN & TEST SET BATCH GENERATORS----------------------
    train_iterator = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    test_iterator  = DataLoader(dataset=test_set,  batch_size=batch_size, shuffle=True, num_workers=2)
    
    # EPOCH LOOPS--------------------------------------------
    listO_models = []
    listO_train_losses, listO_test_losses = [], []
    listO_train_acc, listO_test_acc = [], []
    c1, c2 = 'print', 'print'
    for epoch in range(n_epochs):
        model.to(device)
        
        # TRAIN_LOOPS-----------------------------------------
        n_matches_train = 0
        iter_loss_train = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            
            movies  = Variable(batch[0][:, 0]).to(device)
            users   = Variable(batch[0][:, 1]).to(device)
            ratings = Variable(batch[1]).to(device)                               # make data diffrentiable
            
            optimizer.zero_grad()                                                 # Init
            y_pred = model(users, movies)                                         # Predict
            loss_train = criterion(y_pred, ratings)                               # Calculate loss
            iter_loss_train += loss_train.data.item()                             # Accumulate the loss
            loss_train.backward()                                                 # Backprop
            optimizer.step()                                                      # Update weights
            
            if c1 == 'print':
                print('TRAINING >>>>')                                            # confirmation!!
                c1 = 'no_print' 
            
            y_pred_scalar = retrn_scalar_y_pred(y_pred)
            n_matches_train += retrn_n_matches(y_pred_scalar, ratings)
       
        # COLLECT TRAIN ACCURACIES & LOSSES--------------------    
        train_acc = n_matches_train/len(train_set) 
        listO_train_acc.append(train_acc)
        train_loss = iter_loss_train/(batch_idx+1)
        listO_train_losses.append(train_loss)
        
        # COLLECT MODELS---------------------------------------
        listO_models.append(model)
        
        # EVAL_LOOPS-------------------------------------------
        n_matches_test = 0
        iter_loss_test = 0.0
        model.eval()
        with torch.no_grad():  
            for batch_idx, batch in enumerate(test_iterator):
                
                movies  = Variable(batch[0][:, 0]).to(device)
                users   = Variable(batch[0][:, 1]).to(device)
                ratings = Variable(batch[1]).to(device)  
                
                y_pred = model(users, movies)                         # Predict
                loss_test = criterion(y_pred, ratings)                # Calculate loss
                iter_loss_test += loss_test.data.item() 
                
                if c2 == 'print':
                    print('EVALUATING >>>>')                          # confirmation!!
                    c2 = 'no_print' 
                
                y_pred_scalar = retrn_scalar_y_pred(y_pred)
                n_matches_test += retrn_n_matches(y_pred_scalar, ratings)
        
        # COLLECT TEST ACCURACIES & LOSSES---------------------
        test_acc = n_matches_test/len(test_set)
        listO_test_acc.append(test_acc)
        test_loss = iter_loss_test/(batch_idx+1)
        listO_test_losses.append(test_loss)

        # DISPLAY-----------------------------------------------
        params = [epoch+1, loss_train, loss_test, train_acc, test_acc]
        print('Epoch: {:03}  train_loss: {:.3f}  |  test_loss: {:.3f}  |  train_acc: {:.4f}  |  test_acc: {:.4f}'.format(*params))
        
    print()
    print('RETURNED: listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models')
    print()
    return listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models

# MODEL_1

In [0]:
class Recommender_Classify_1(torch.nn.Module):
    
    def __init__(self, n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes):
        super().__init__()
        
	     # create user & item embeddings of same size:
        self.user_embeddings = nn.Embedding(n_users, embedding_dim, sparse=False)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim, sparse=False)
        
        self.weights1 = nn.Linear(embedding_dim*2, hidden_dim)
        self.weights2 = nn.Linear(hidden_dim, n_classes)
        
        self.batch_norm1 = nn.BatchNorm1d(embedding_dim*2)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)
        
        self.dropout1 = nn.Dropout2d(p=dropout1)
        self.dropout2 = nn.Dropout2d(p=dropout2)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, user, item):
        
    	# CREATING USER LATENT VARIABLES
        
        # embedding
        user_vec = self.user_embeddings(user)
        item_vec = self.item_embeddings(item)
        
        # representation of iput(concat user & item vectors):
        input_vec = torch.cat((user_vec, item_vec), dim = 1)
                              
        # layer 1
        hidden = self.batch_norm1(input_vec) 
        hidden = self.dropout1(hidden) 
        hidden = self.weights1(hidden) 
        hidden = self.relu(hidden)
        # layers 2 
        hidden = self.batch_norm2(hidden) 
        hidden = self.dropout2(hidden) 
        hidden = self.weights2(hidden) 
        
        y_pred     = self.softmax(hidden)
        return y_pred

### TRIAL 1:

In [0]:
n_users, n_items, embedding_dim, hidden_dim, n_classes = 50000, 5000, 50, 50, 5
dropout1, dropout2 = 0.5, 0.25

model = Recommender_Classify_1(n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

train_set = train_dataset
test_set = test_dataset
batch_size = 1000                                                  
n_epochs = 30

%time listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models = fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs)

TRAINING >>>>
EVALUATING >>>>
Epoch: 001  train_loss: 1.504  |  test_loss: 1.506  |  train_acc: 0.3562  |  test_acc: 0.3903
Epoch: 002  train_loss: 1.490  |  test_loss: 1.478  |  train_acc: 0.3930  |  test_acc: 0.4215
Epoch: 003  train_loss: 1.442  |  test_loss: 1.449  |  train_acc: 0.4160  |  test_acc: 0.4361
Epoch: 004  train_loss: 1.444  |  test_loss: 1.446  |  train_acc: 0.4293  |  test_acc: 0.4433
Epoch: 005  train_loss: 1.450  |  test_loss: 1.407  |  train_acc: 0.4381  |  test_acc: 0.4486
Epoch: 006  train_loss: 1.410  |  test_loss: 1.453  |  train_acc: 0.4453  |  test_acc: 0.4517
Epoch: 007  train_loss: 1.414  |  test_loss: 1.417  |  train_acc: 0.4504  |  test_acc: 0.4543
Epoch: 008  train_loss: 1.459  |  test_loss: 1.405  |  train_acc: 0.4548  |  test_acc: 0.4561
Epoch: 009  train_loss: 1.450  |  test_loss: 1.435  |  train_acc: 0.4585  |  test_acc: 0.4570
Epoch: 010  train_loss: 1.427  |  test_loss: 1.462  |  train_acc: 0.4613  |  test_acc: 0.4586
Epoch: 011  train_loss: 1.437 

### TRIAL 2:

In [0]:
n_users, n_items, embedding_dim, hidden_dim, n_classes = 50000, 5000, 50, 50, 5
dropout1, dropout2 = 0, 0

model = Recommender_Classify_1(n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

train_set = train_dataset
test_set = test_dataset
batch_size = 1000                                                  
n_epochs = 30

%time listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models = fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs)

TRAINING >>>>
EVALUATING >>>>
Epoch: 001  train_loss: 1.445  |  test_loss: 1.423  |  train_acc: 0.3849  |  test_acc: 0.4170
Epoch: 002  train_loss: 1.437  |  test_loss: 1.442  |  train_acc: 0.4414  |  test_acc: 0.4429
Epoch: 003  train_loss: 1.434  |  test_loss: 1.412  |  train_acc: 0.4645  |  test_acc: 0.4501
Epoch: 004  train_loss: 1.426  |  test_loss: 1.426  |  train_acc: 0.4756  |  test_acc: 0.4530
Epoch: 005  train_loss: 1.430  |  test_loss: 1.455  |  train_acc: 0.4836  |  test_acc: 0.4542
Epoch: 006  train_loss: 1.419  |  test_loss: 1.461  |  train_acc: 0.4893  |  test_acc: 0.4543
Epoch: 007  train_loss: 1.412  |  test_loss: 1.450  |  train_acc: 0.4947  |  test_acc: 0.4542
Epoch: 008  train_loss: 1.381  |  test_loss: 1.427  |  train_acc: 0.4996  |  test_acc: 0.4541
Epoch: 009  train_loss: 1.393  |  test_loss: 1.426  |  train_acc: 0.5042  |  test_acc: 0.4539
Epoch: 010  train_loss: 1.371  |  test_loss: 1.446  |  train_acc: 0.5087  |  test_acc: 0.4535
Epoch: 011  train_loss: 1.368 

# ROUGH WORK :

### CROSS ENTROPY LOSS

In [0]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([2, 4, 0])

In [0]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)


In [0]:
input

tensor([[-0.4187, -0.0201, -0.8396,  1.0466,  0.6363],
        [ 0.4213,  0.4998, -0.6429, -0.1832,  1.4841],
        [-0.2525, -1.7729,  0.9540,  2.0658,  1.0311]], requires_grad=True)

In [0]:
target

tensor([0, 3, 4])

In [0]:
output = loss(input, target)
output.backward()
output

tensor(2.1118, grad_fn=<NllLossBackward>)

### CHECKING COMPATIBILITY  BATCH FORMAT, MODEL PREDICTION & LOSS FN:

In [0]:
model#.to(device)
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
criterion = criterion#.to(device)

In [0]:
train_iterator = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True, num_workers=2)
for batch_idx, batch in enumerate(train_iterator):

        movies  = Variable(batch[0][:, 0])#.to(device)
        users   = Variable(batch[0][:, 1])#.to(device)
        ratings = Variable(batch[1])#.to(device)                               

        optimizer.zero_grad()                                                 
        y_pred = model(users, movies) 
        
        loss_train = criterion(y_pred, ratings)   
        if batch_idx == 0: break

In [0]:
y_pred

tensor([[0.2418, 0.0957, 0.3182, 0.2439, 0.1004],
        [0.1615, 0.1615, 0.1833, 0.3323, 0.1615]], grad_fn=<SoftmaxBackward>)

In [0]:
ratings

tensor([2, 0])

In [0]:
loss_train 

tensor(1.5727, grad_fn=<NllLossBackward>)